In [10]:
from icecream import ic
import os
import numpy as np
import pandas as pd
from nltk.stem import PorterStemmer 

stemmer = PorterStemmer()
dictionary = []

dataset_path = "/Users/mohamad/Documents/GitHub/BrainQuest/data/documents/Cranfield collection/cran.all.1400"

# read the cran.all.1400 file that contain the all of the datasets
try:
    with open(dataset_path, "r") as f:
        raw_dataset = f.read()
except IOError:
    print("The file does not exist, or the entered path is invalid!")

raw_dataset = raw_dataset.split(".I") # split the docs by index
raw_dataset.remove(raw_dataset[0]) # remove the empty first index

doc_index = 0
for doc in raw_dataset:
    doc_index += 1
    doc_path = "/Users/mohamad/Documents/GitHub/BrainQuest/data/documents/" + "/doc" + str(doc_index) + ".txt"
    if os.path.exists(doc_path): # if file exist ignore
        pass
    else:
        with open(doc_path, "w") as document: # write each index into the separated doc
            document.write(doc)

# read the list of stop words
stopword_file_path = "/Users/mohamad/Documents/GitHub/BrainQuest/data/stopword/words.txt"
with open(stopword_file_path, "r") as file:
    stop_words = file.read()
    
# separate the doc section from the document
def transfer_to_list(doc):
    start = doc.find(".W")
    doc = doc[start + 2:]
    doc = doc.split()
    return doc

# stemming
stemmer = PorterStemmer()
dictionary = []
def create_dictionary(doc, dictionary):
    for element in doc:
        if element.isalpha():
            stem_word = stemmer.stem(element)
            if (stem_word not in stop_words) and (stem_word not in dictionary):
                dictionary.append(stem_word)
        
# read each doc and pass them through the methods
for num in range(1, doc_index + 1):
    doc_path = "/Users/mohamad/Documents/GitHub/BrainQuest/data/documents" + "/doc" + str(num) + ".txt"
    with open(doc_path, "r") as file:
        doc = file.read()
        doc = transfer_to_list(doc)
        create_dictionary(doc, dictionary)
        
        
        


# initializing the numpy arrays
tf_array = np.zeros((len(dictionary), doc_index))
idf_array = np.zeros((len(dictionary), 1))
tf_idf_array = np.zeros((len(dictionary), doc_index))


def calculate_term_frequency(doc, dictionary, doc_no):
    for word in dictionary:
        if word in doc:
            frequency = doc.count(word)
            row = dictionary.index(word)
            column = doc_no - 1
            tf_array[row, column] += frequency


def calculate_inverse_document_frequency(doc, dictionary):
    for word in dictionary:
        if word in doc:
            row = dictionary.index(word)
            idf_array[row, 0] += 1
    
    
# extract the text from each document
def extract_the_text(doc):
    start = doc.find(".W")
    doc = doc[start + 2:]
    return doc

doc_names_list = []
for num in range(1, doc_index + 1):
    doc_path = "/Users/mohamad/Documents/GitHub/BrainQuest/data/documents" + "/doc" + str(num) + ".txt"
    doc_names_list.append("doc" + str(num) + ".txt")
    with open(doc_path, "r") as file:
        doc = file.read()
        doc = extract_the_text(doc)
        calculate_term_frequency(doc, dictionary, num)
        calculate_inverse_document_frequency(doc, dictionary)

# calculating tf array
row, column = np.shape(tf_array)
for r in range(row):
    for c in range(column):
        if tf_array[r, c] > 0:
            tf_array[r, c] = 1 + np.log10([tf_array[r, c]])
        else:
            tf_array[r, c] = 1

count_of_documents = column
row, column = np.shape(idf_array)


# calculating idf array
for r in range(row):
    if idf_array[r, 0] > 0:
        idf_array[r, 0] = np.log10(count_of_documents / idf_array[r, 0])
    else:
        idf_array[r, 0] = 0


/var/folders/jj/f2yw3cln5174grx76c3nskwh0000gn/T/ipykernel_50197/2333235116.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tf_array[r, c] = 1 + np.log10([tf_array[r, c]])


In [11]:

def create_dictionary(doc, dictionary):
    for element in doc:
        if element.isalpha():
            stem_word = stemmer.stem(element)
            if (stem_word not in stop_words) and (stem_word not in dictionary):
                dictionary.append(stem_word)


# multiply peer to peer each row of tf-array in idf-array
row, column = np.shape(tf_idf_array)
for r in range(row):
    for c in range(column):
        tf_idf_array[r, c] = tf_array[r, c] * idf_array[r, 0]


# function to create the excel files
def create_excel(input , name, columns, index):
    df = pd.DataFrame(input, columns = columns, index = index) # Create a pandas dataframe
    df.to_excel(name) # Create an excel file
    print(df)
    

create_excel(tf_array, "tf_excel.xlsx", doc_names_list, dictionary)
create_excel(idf_array, "idf_excel.xlsx", ["IDF"], dictionary)
create_excel(tf_idf_array, "tf_idf_excel.xlsx", doc_names_list, dictionary)


            doc1.txt  doc2.txt  doc3.txt  doc4.txt  doc5.txt  doc6.txt  \
experiment  1.477121       1.0       1.0       1.0       1.0       1.0   
investig    1.000000       1.0       1.0       1.0       1.0       1.0   
aerodynam   1.000000       1.0       1.0       1.0       1.0       1.0   
slipstream  1.698970       1.0       1.0       1.0       1.0       1.0   
studi       1.000000       1.0       1.0       1.0       1.0       1.0   
...              ...       ...       ...       ...       ...       ...   
squir       1.000000       1.0       1.0       1.0       1.0       1.0   
standoff    1.000000       1.0       1.0       1.0       1.0       1.0   
hoshizaki   1.000000       1.0       1.0       1.0       1.0       1.0   
recover     1.000000       1.0       1.0       1.0       1.0       1.0   
fralich     1.000000       1.0       1.0       1.0       1.0       1.0   

            doc7.txt  doc8.txt  doc9.txt  doc10.txt  ...  doc1391.txt  \
experiment       1.0       1.0  1.0000